In [1]:
import numpy as np
import pandas as pd


In [2]:
hmd = pd.read_csv("./hmd_weekly_deaths/stmf.csv") #mortality dataset
coviddb = pd.read_csv("owid-covid-data.csv") #covid dataset

#Make a list of the country codes from the coviddb that appear in the hmd
country_codes = ['AUS', 'AUT', 'BEL', 'BGR', 'HRV', 'CHL', 'CZE', 'DNK', 'GBR', 'EST',
 'FIN', 'FRA', 'DEU', 'GRC', 'HUN', 'ISL', 'ISR', 'ITA', 'LVA', 'LTU', 'LUX', 'NLD',
  'NZL', 'NOR', 'POL', 'PRT', 'KOR', 'RUS', 'SVN', 'SVK', 'ESP', 'CHE', 'SWE', 'TWN', 'USA']

coviddb_selected = coviddb[coviddb.iso_code.isin(country_codes)] #filter out unmatching countries

#Trim data to useful attributes
coviddb_selected=coviddb_selected[['iso_code','date', 'total_cases','new_cases', 'total_deaths', 'new_deaths', 'total_cases_per_million', 'new_cases_per_million', 'hosp_patients', 'population']]

coviddb_selected['date']=pd.to_datetime(coviddb_selected['date']) #convert date column to datetime

#convert country to category
coviddb_selected.iso_code.astype('category')

#resample from daily to weekly
coviddb_selected = coviddb_selected.reset_index().groupby('iso_code', as_index=False).resample('w', on = 'date').\
agg({'total_cases':'first', 'new_cases':'sum','total_deaths':'first','new_deaths':'sum','total_cases_per_million':'first', 'new_cases_per_million':'sum',\
'hosp_patients':'first','population':'first'}).reset_index()

#restore columns
coviddb_selected.rename(columns={'level_0':'country'}, inplace = True)


C:\Users\melni\AppData\Local\Temp\ipykernel_23344\249112126.py:1: DtypeWarning: Columns (1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  hmd = pd.read_csv("./hmd_weekly_deaths/stmf.csv") #mortality dataset


In [3]:
#Country codes have changed to integers; this changes them back to country codes
coviddb_selected.country.astype('int64')
country_index = np.arange(0, 35, 1)
country_dict = dict(zip(country_index, country_codes))
coviddb_selected = coviddb_selected.replace({"country": country_dict})

In [9]:
week = coviddb_selected['date'].dt.week
year = coviddb_selected['date'].dt.year
coviddb_selected['week'] = week
coviddb_selected['year'] = year

C:\Users\melni\AppData\Local\Temp\ipykernel_23344\1723132619.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  week = coviddb_selected['date'].dt.week


In [11]:
#Reorder columns and remove original date
cols = ['country', 'year', 'week', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'total_cases_per_million', 'hosp_patients', 'population']
coviddb_selected = coviddb_selected[cols]

In [15]:
coviddb_selected

,country,year,week,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,hosp_patients,population
0,AUS,2020,4,4.0,4.0,NaN,0.0,0.155,NaN,25788217.0
1,AUS,2020,5,5.0,8.0,NaN,0.0,0.194,NaN,25788217.0
2,AUS,2020,6,12.0,3.0,NaN,0.0,0.465,NaN,25788217.0
3,AUS,2020,7,15.0,0.0,NaN,0.0,0.582,NaN,25788217.0
4,AUS,2020,8,15.0,0.0,NaN,0.0,0.582,NaN,25788217.0
...,...,...,...,...,...,...,...,...,...,...
3651,USA,2022,1,56345834.0,5064970.0,830299.0,11876.0,169249.873,108779.0,332915074.0
3652,USA,2022,2,61703251.0,5614161.0,842288.0,13542.0,185342.316,142248.0,332915074.0
3653,USA,2022,3,66610143.0,4951791.0,854838.0,14705.0,200081.487,150906.0,332915074.0
3654,USA,2022,4,71802940.0,3561887.0,870600.0,16801.0,215679.450,144756.0,332915074.0


In [13]:
#export work so far
coviddb_selected.to_csv('selected_covid_weekly')